In [1]:
import pandas as pd
import matplotlib as pt
import alpaca_trade_api as tradeapi
import requests
import yfinance as yf
from datetime import datetime, timedelta

In [2]:
# Alpaca API config stuff

APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'
APCA_MARKET_DATA = 'https://data.alpaca.markets/v1'
APCA_API_KEY = 'PKDQTHU4Z3R8ZTXFOGXN'
APCA_SECRET_KEY = 'FBE3WStDzqd6WFATRUVQbDLRlht0E6p7kO6gLOdB'

api = tradeapi.REST(APCA_API_KEY, APCA_SECRET_KEY, APCA_API_BASE_URL)

In [3]:
# IEX API config stuff

IEX_CLOUD_API_KEY = 'Tpk_12e6eee6ed7d4026a0a87dee063b86bd'
IEX_CLOUD_SECRET_KEY = 'Tsk_620c4eafd98f479f990e532c3a0b2cb0'
SANDBOX_BASE_URL = 'https://sandbox.iexapis.com/stable/'

In [4]:
# 50-day SMA

def fifty_day_sma(ticker):
    FIFTY = 50.
    curr_date = str(datetime.now()).split()[0]
    year_ago = str(datetime.now() - timedelta(days=365)).split()[0]
    ticker_info = yf.Ticker(ticker)
 
    one_year_price_history = ticker_info.history(start=year_ago, end=curr_date)
    fifty_day_moving_average = one_year_price_history.iloc[-50:]['Close'].sum() / FIFTY
    return fifty_day_moving_average

In [5]:
# 200-day SMA

def two_hundred_day_sma(ticker):
    TWO_HUNDRED = 200.
    curr_date = str(datetime.now()).split()[0]
    year_ago = str(datetime.now() - timedelta(days=365)).split()[0]
    ticker_info = yf.Ticker(ticker)

    one_year_price_history = ticker_info.history(start=year_ago, end=curr_date)
    two_hundred_day_moving_average = one_year_price_history.iloc[-200:]['Close'].sum() / TWO_HUNDRED
    return two_hundred_day_moving_average

In [6]:
# checks if there is a Golden Cross (50-day SMA crosses 200-day SMA)

def golden_cross(ticker):
    fifty_sma = fifty_day_sma(ticker)
    two_hundred_sma = two_hundred_day_sma(ticker)
    
    if fifty_sma > two_hundred_sma:
        return True
    return False

In [7]:
# test SMAs + Golden Cross

print(fifty_day_sma('MSFT'), two_hundred_day_sma('MSFT'), golden_cross('MSFT'))

214.37353790283203 197.5154337310791 True


In [72]:
# calculates 14-day MFI for specified stock

def mfi(ticker):
    
    # extract last 14 days' worth of price info
    curr_date = str(datetime.now()).split()[0]
    fifty_days_ago = str(datetime.now() - timedelta(days=50)).split()[0]
    ticker_info = yf.Ticker(ticker)
    one_year_price_history = ticker_info.history(start=fifty_days_ago, end=curr_date)
    two_week_ph = one_year_price_history[-14:]
    
    two_week_ph = two_week_ph.assign(TP = lambda tp:(two_week_ph['High'] + two_week_ph['Low'] + two_week_ph['Close']) / 3.)
    
    two_week_ph = two_week_ph.assign(RMF = lambda rmf: two_week_ph['TP'] * two_week_ph['Volume'])

    positive_rmf = 0
    negative_rmf = 0
    
    for i in range(1, len(two_week_ph['TP'])):
        if two_week_ph['TP'].iloc[i] > two_week_ph['TP'].iloc[i - 1]:
#             tp_diff.append(1)
            positive_rmf += two_week_ph['RMF'].iloc[i]
        else:
#             tp_diff.append(-1)
            negative_rmf += two_week_ph['RMF'].iloc[i]
        
#     two_week_ph = two_week_ph.assign(TP_Diff = [1 if x == 1 else -1 for x in tp_diff])
    
    mf_ratio = positive_rmf / negative_rmf
    
    money_flow_index = 100 - (100 / (1 + mf_ratio))
    
#     return two_week_ph
    return mf_ratio, money_flow_index

In [73]:
# print(mfi('AAPL'), mfi('MSFT'), mfi('TGT'), mfi('WDAY'), mfi('GME'))
mfi('MSFT')

(1.326851785700906, 57.02347669304708)

In [74]:
mfi('WDAY')

(1.8095159674742833, 64.40668031159166)

In [75]:
mfi('AAPL')

(1.0918834117272838, 52.19618864063305)

In [76]:
mfi('GME')

(2.624021421358502, 72.40634412074917)